Material courtesy of Joseph Near, University of Vermont

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import pandas as pd
import numpy as np

adult = pd.read_csv("adult_with_pii.csv")
def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)
def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

# The Exponential Mechanism

The fundamental mechanisms we have seen so far (Laplace and Gaussian) are focused on numerical answers, and add noise directly to the answer itself. What if we want to return a precise answer (i.e. no added noise), but still preserve differential privacy? One solution is the exponential mechanism, which allows selecting the "best" element from a set while preserving differential privacy. The analyst defines which element is the "best" by specifying a *scoring function* that outputs a score for each element in the set, and also defines the set of things to pick from. The mechanism provides differential privacy by *approximately* maximizing the score of the element it returns - in other words, to satisfy differential privacy, the exponential mechanism sometimes returns an element from the set which does *not* have the highest score.

The exponential mechanism satisfies $\epsilon$-differential privacy:

1. The analyst selects a set $\mathcal{R}$ of possible outputs
2. The analyst specifies a scoring function $u : \mathcal{D} \times \mathcal{R} \rightarrow \mathbb{R}$ with global sensitivity $\Delta u$
3. The exponential mechanism outputs $r \in \mathcal{R}$ with probability proportional to:

\begin{align}
\exp \Big(\frac{\epsilon u(x, r)}{2 \Delta u} \Big)
\end{align}

The biggest practical difference between the exponential mechanism and the previous mechanisms we've seen (e.g. the Laplace mechanism) is that the output of the exponential mechanism is *always* a member of the set $\mathcal{R}$. This is extremely useful when selecting an item from a finite set, when a noisy answer would not make sense. For example, we might want to pick a date for a big meeting, which uses each participant's personal calendar to maximize the number of participants without a conflict, while providing differential privacy for the calendars. Adding noise to a date doesn't make much sense: it might turn a Friday into a Saturday, and *increase* the number of conflicts significantly. The exponential mechanism is perfect for problems like this one: it selects a date *without noise*.

The exponential mechanism is interesting for several reasons:

- The privacy cost of the mechanism is just $\epsilon$, regardless of the size of $\mathcal{R}$ - more on this next.
- It works for both finite and infinite sets $\mathcal{R}$, but it can be really challenging to build a practical implementation which samples from the appropriate probability distribution when $\mathcal{R}$ is infinite.
- It represents a "fundamental mechanism" of $\epsilon$-differential privacy: all other $\epsilon$-differentially private mechanisms can be defined in terms of the exponential mechanism with the appropriate definition of the scoring function $u$.

## The Exponential Mechanism for Finite Sets



What is the most common marital status in the dataset?

In [9]:
options = adult['Martial Status'].unique()

print(options)

def score(data, option):
    return data.value_counts()[option]/1000

print(score(adult['Martial Status'], 'Never-married'))
print(score(adult['Martial Status'], 'Married-civ-spouse'))
print(score(adult['Martial Status'], 'Divorced'))
print(score(adult['Martial Status'], 'Married-spouse-absent'))
print(score(adult['Martial Status'], 'Separated'))
print(score(adult['Martial Status'], 'Married-AF-spouse'))
print(score(adult['Martial Status'], 'Widowed'))

['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Married-AF-spouse' 'Widowed']
10.683
14.976
4.443
0.418
1.025
0.023
0.993


In [15]:
def exponential(x, R, u, sensitivity, epsilon):
    # Calculate the score for each element of R
    scores = [u(x, r) for r in R]
    
    # Calculate the probability for each element, based on its score
    probabilities = [np.exp(epsilon * score / (2 * sensitivity)) for score in scores]
    
    # Normalize the probabilties so they sum to 1
    probabilities = probabilities / np.linalg.norm(probabilities, ord=1)

    # Choose an element from R based on the probabilities
    return np.random.choice(R, 1, p=probabilities)[0]

exponential(adult['Martial Status'], options, score, 1, 1)

'Married-civ-spouse'

In [16]:
r = [exponential(adult['Martial Status'], options, score, 1, 1) for i in range(200)]
pd.Series(r).value_counts()

Married-civ-spouse    176
Never-married          23
Divorced                1
dtype: int64